dicease classification model: input symptoms and output diceases accordingly

In [ ]:
!pip install -q streamlit
!pip install -q joblib

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.8/9.8 MB 21.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 29.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 5.3 MB/s eta 0:00:00


In [ ]:
# Import libraries
import os
import pandas as pd
from transformers import BertTokenizerFast, BertForSequenceClassification, Trainer, TrainingArguments
import torch
from torch.utils.data import Dataset
#import pdfplumber
#from langchain.text_splitter import RecursiveCharacterTextSplitter
#from langchain.vectorstores import FAISS
#from langchain.embeddings import HuggingFaceEmbeddings
#from langchain.docstore.document import Document
from sklearn.metrics import precision_score, recall_score, f1_score
import numpy as np
import gc
import glob
from collections import Counter
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

import joblib

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
# Define file path
DATA_PATH = "/content/gdrive/MyDrive/processed_diseases-2400_NER.csv"

In [ ]:
# Load CSV
try:
    df = pd.read_csv(
        DATA_PATH,
        usecols=["Disease", "Treatment", "Symptoms"],
        engine="pyarrow",
        dtype_backend="numpy_nullable",
        dtype={"Disease": "string", "Treatment": "string", "Symptoms": "string"}
    ).rename(columns={"Disease": "disease_name", "Treatment": "treatment", "Symptoms": "symptoms"})
    print(f"Loaded CSV with {len(df)} rows")
except Exception as e:
    raise ValueError(f"Error loading CSV: {e}")

Loaded CSV with 2399 rows


In [ ]:
# Verify columns
required_cols = ["symptoms", "disease_name", "treatment"]
if not all(col in df.columns for col in required_cols):
    raise ValueError(f"Missing columns: {[col for col in required_cols if col not in df.columns]}")

In [ ]:
# Define binary labels (treatment present/absent)
df["label"] = [1 if pd.notna(row["treatment"]) and row["treatment"].strip() != "" else 0 for _, row in df.iterrows()]
num_labels = 2  # Binary classification
print(f"Label distribution: \n{df['label'].value_counts()}")

Label distribution: 
label
1    2399
Name: count, dtype: int64


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2399 entries, 0 to 2398
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   disease_name  2399 non-null   string
 1   treatment     2399 non-null   string
 2   symptoms      2399 non-null   string
 3   label         2399 non-null   int64 
dtypes: int64(1), string(3)
memory usage: 75.1 KB


In [ ]:
df.head(5)

,disease_name,treatment,symptoms,label
0,Vulvodynia,"Topical lidocaine, amitriptyline (1050 mg dail...","Chronic vulvar burning, stinging, rawness; pai...",1
1,Cold Sores,"Acyclovir cream (5%, 5x daily), valacyclovir (...","Lip tingling, painful blisters, crusting; mild...",1
2,Renal Cell Carcinoma,"Nephrectomy, sunitinib (50 mg daily), nivoluma...","Hematuria, flank pain, weight loss, fatigue, a...",1
3,Scabies,"Permethrin 5% cream, ivermectin (200 mcg/kg), ...","Nighttime itching, red rash, skin burrows.",1
4,Parkinson’s Disease,"Levodopacarbidopa (25/100 mg 3x daily), pramip...","Resting tremors, bradykinesia, rigidity, postu...",1


In [ ]:
# Define possible delimiters
delimiters = [',', ';', '|', '\t', ' ']

# Function to clean and split text while keeping delimiters as tokens
def clean_and_split_with_delimiters(text):
    if pd.isna(text):
        return ''

    # Treat delimiters as special tokens
    for delimiter in delimiters:
        text = text.replace(delimiter, f" | ")

    return text.lower()  # Convert text to lowercase to standardize

# Apply the new cleaning function
df['cleaned_symptoms'] = df['symptoms'].apply(clean_and_split_with_delimiters)

# Convert text to numerical features using TF-IDF
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(df['cleaned_symptoms'])
y = df['disease_name']

#comment out so i can train the whole dataset
# # Split into training and testing sets (80% train, 20% test)
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train the model
clf = LogisticRegression(
    penalty='l2',          # Ridge (L2) regularization to prevent overfitting
    C=1.0,                 # Regularization strength (default=1.0, lower values increase regularization)
    solver='liblinear',     # Suitable for small datasets and works well with sparse data
    max_iter=200,           # Sufficient iterations for convergence
    class_weight='balanced') # Handles class imbalances by adjusting weights)
clf.fit(X, y)

# # Evaluate on training data
# y_train_pred = clf.predict(X_train)
# train_accuracy = accuracy_score(y_train, y_train_pred)

# # Evaluate on test data
# y_test_pred = clf.predict(X_test)

# import matplotlib.pyplot as plt
# import numpy as np

# # Accuracy scores
# train_accuracy = accuracy_score(y_train, y_train_pred)
# test_accuracy = accuracy_score(y_test, y_test_pred)

# # Bar chart
# plt.figure(figsize=(6, 4))
# plt.bar(["Train Accuracy", "Test Accuracy"], [train_accuracy, test_accuracy], color=['blue', 'orange'])
# plt.ylim(0, 1)
# plt.ylabel("Accuracy")
# plt.title("Training vs. Test Accuracy")
# plt.show()

# Function to Predict Top 3 Approximate Diseases (with delimiter retention)
def predict_top_5_diseases(symptom_input):
    # Clean and process input while keeping delimiters as tokens
    cleaned_input = clean_and_split_with_delimiters(symptom_input)

    # # Convert input to TF-IDF representation
    input_vector = vectorizer.transform([cleaned_input])

    # # Get predicted probabilities for all diseases
    probas = clf.predict_proba(input_vector)[0]

    # # Pair disease names with their probabilities
    disease_probabilities = list(zip(clf.classes_, probas))

    #Sort by probas which is second element in disease_probabilities which is a list data type
    sorted_disease = sorted(disease_probabilities , key=lambda x: x[1], reverse = True)

    #Sort diseases by probability in descending order and get top 5
    top_5_diseases = sorted(sorted_disease, key=lambda x: x[1], reverse=True)[:5]

    return top_5_diseases

In [ ]:
#Test example with users inputing fever and rash symptoms
cleaned_input = clean_and_split_with_delimiters("fever,rash")

# # Convert input to TF-IDF representation
input_vector = vectorizer.transform([cleaned_input])

# # Get predicted probabilities for all diseases
probas = clf.predict_proba(input_vector)[0]

# # Pair disease names with their probabilities
disease_probabilities = list(zip(clf.classes_, probas))

#Sort by probas which is second element in disease_probabilities which is a list data type
sorted_disease = sorted(disease_probabilities , key=lambda x: x[1], reverse = True)

#Sort diseases by probability in descending order and get top 5
top_5_diseases = sorted(sorted_disease, key=lambda x: x[1], reverse=True)[:5]

top_5_diseases

[('EpsteinBarr Virus (2)', np.float64(0.0008540485569227423)),
 ('Enterovirus (NonPolio Enterovirus Infection)',
  np.float64(0.0008126788050867763)),
 ('Lyme Disease', np.float64(0.0007892462361763946)),
 ('Herpes Zoster Meningitis (Entry 1)', np.float64(0.0007854519080706658)),
 ('Autoinflammatory Disease (General)', np.float64(0.0007832082665867253))]

In [ ]:
#Test example with users inputing fever and headache symptoms
cleaned_input = clean_and_split_with_delimiters("headache,fever")

  # # Convert input to TF-IDF representation
input_vector = vectorizer.transform([cleaned_input])

  # # Get predicted probabilities for all diseases
probas = clf.predict_proba(input_vector)[0]

  # # Pair disease names with their probabilities
disease_probabilities = list(zip(clf.classes_, probas))

  #Sort by probas which is second element in disease_probabilities which is a list data type
sorted_disease = sorted(disease_probabilities , key=lambda x: x[1], reverse = True)

  #Sort diseases by probability in descending order and get top 5
top_5_diseases = sorted(sorted_disease, key=lambda x: x[1], reverse=True)[:5]

top_5_diseases

[('Viral Meningitis in Children', np.float64(0.0008359303322579575)),
 ('Viral Encephalitis', np.float64(0.0008275084567961582)),
 ('CNS Infection (General)', np.float64(0.0008275084567961575)),
 ('Viral Meningitis (Duplicate, Variation)',
  np.float64(0.0007783366766052738)),
 ('Herpes Zoster Meningitis (Entry 1)', np.float64(0.0007727735774739741))]

In [ ]:
# Final testing for the function to apply the model inside the function when user enters symptoms
user_symptoms = "Fever, rash"
predicted_top_5_diseases = predict_top_5_diseases(user_symptoms)

# Display the top 5 ranked diseases and corresponding treatment column from dataframe df as well
predicted_top_5_diseases

[('EpsteinBarr Virus (2)', np.float64(0.0008540485569227423)),
 ('Enterovirus (NonPolio Enterovirus Infection)',
  np.float64(0.0008126788050867763)),
 ('Lyme Disease', np.float64(0.0007892462361763946)),
 ('Herpes Zoster Meningitis (Entry 1)', np.float64(0.0007854519080706658)),
 ('Autoinflammatory Disease (General)', np.float64(0.0007832082665867253))]

In [ ]:
#my next step is to append the diseases back to the dataframe to get the correspoinding treatment col

In [ ]:
#deploy the model to app
#Save the model and vectorizer
# 🔹 Set Google Drive path (Modify if necessary)
drive_path = "/content/gdrive/MyDrive/"

joblib.dump(clf, os.path.join(drive_path, 'disease_model.pkl'))
joblib.dump(vectorizer, os.path.join(drive_path, 'tfidf_vectorizer.pkl'))

['/content/gdrive/MyDrive/tfidf_vectorizer.pkl']

In [ ]:
%%writefile app.py
import streamlit as st
import pandas as pd
import joblib
import os


# 🔹 Set Google Drive path (Modify if necessary)
drive_path = "/content/gdrive/MyDrive/"

# 🔹 Function to load the trained model and vectorizer
@st.cache_resource
def load_model():
    model_path = os.path.join(drive_path, "disease_model.pkl")
    vectorizer_path = os.path.join(drive_path, "tfidf_vectorizer.pkl")

    if not os.path.exists(model_path) or not os.path.exists(vectorizer_path):
        st.error("🚨 Model or vectorizer not found! Please train and save them first.")
        return None, None

    clf = joblib.load(model_path)
    vectorizer = joblib.load(vectorizer_path)
    return clf, vectorizer

# 🔹 Load Model & Vectorizer
clf, vectorizer_tfidf = load_model()

# 🔹 Load Dataset for Symptom Selection
@st.cache_data
def load_data():
    data_path = "/content/gdrive/MyDrive/processed_diseases-2400_NER.csv"
    return pd.read_csv(data_path)

df = load_data()

# ✅ Streamlit App UI
st.title("🔬 Disease Prediction App")
st.write("**Select symptoms and predict the most likely disease.**")

# 🔹 Extract Unique Symptoms
all_symptoms = set()
for symptoms in df["Symptoms"].astype(str).fillna(""):
    all_symptoms.update(symptoms.split(", "))


# 🔹 User Input: Multi-select Symptoms
selected_symptoms = st.multiselect("🩺 Select Symptoms:", sorted(all_symptoms))

# 🔹 Prediction Button
if st.button("🔍 Predict Disease"):
    if not selected_symptoms:
        st.warning("⚠️ Please select at least one symptom.")
    elif clf is None or vectorizer_tfidf is None:
        st.error("🚨 Model is not loaded. Please check the file paths and restart.")
    else:
        # Convert user input to TF-IDF format
        input_text = ", ".join(selected_symptoms)
        input_vectorized = vectorizer_tfidf.transform([input_text])

        # Make Prediction
        prediction = clf.predict(input_vectorized)
        probability = clf.predict_proba(input_vectorized).max()

        # ✅ Display the Prediction
        st.success(f"🩺 **Predicted Disease:** {prediction[0].capitalize()}")
        st.write(f"🔍 **Confidence Score:** {probability:.2f}")

# 🔹 Data Preview
st.write("📊 **Sample Data:**")
st.dataframe(df.head())


Overwriting app.py


In [ ]:
!npm install localtunnel
!streamlit run app.py --server.address=localhost &>/content/logs.txt &
!npx localtunnel --port 8501 & curl https://loca.lt/mytunnelpassword

⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸
added 22 packages in 4s
⠸
⠸3 packages are looking for funding
⠸  run `npm fund` for details
⠸34.46.1.123⠙your url is: https://grumpy-comics-attack.loca.lt
